# Gradient boosting

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline 
import gensim
from gensim import corpora,models,similarities
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
df=pd.read_csv("Mock_data_combined.csv",encoding="unicode_escape")
df.head()
df1=df[["Code Chunck","Indicator"]]
df1["Indicator"]=df['Indicator'].astype(int)

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
def gradientmodel():
    # df.columns.tolist()
    #Text Normalization
    tokenized_case = df1['Code Chunck'].apply(lambda x: x.split())
    bow_vectorizer = CountVectorizer(max_df =0.90,min_df=2,max_features=1000,stop_words='english')
    bow=bow_vectorizer.fit_transform(df1['Code Chunck'])
#     bow.shape
    tfidf_vectorizer = TfidfVectorizer(max_df =0.90,min_df=2,max_features=1000,stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(df1['Code Chunck'])
    wordvec_arrays=np.zeros((len(tokenized_case),100))
    for i in range(len(tokenized_case)):
        wordvec_arrays[i,:]=word_vector(tokenized_case[i],100)
        wordvec_df = pd.DataFrame(wordvec_arrays)
    print(wordvec_df.shape)
    return wordvec_df

In [4]:
def word_vector(tokens,size):
    tokenized_case = df1['Code Chunck'].apply(lambda x: x.split())#tokenizing
    model_w2v = gensim.models.Word2Vec(tokenized_case,size=100,window=5,min_count=2,sg=1,hs=0,negative=10,workers=2,seed=34)
    model_w2v.train(tokenized_case,total_examples = len(df1['Code Chunck']),epochs=20)
#     model_w2v.wv.most_similar(positive='assert')
#     len(model_w2v['assert'])
    vec= np.zeros(size).reshape(1,size)
    count=0.
    for word in tokens:
        try:
            vec +=model_w2v[word].reshape((1,size))
            count+=1.
        except KeyError:
            continue
    if count!=0:
        vec/=count
    return vec       

# gradient boosting

In [5]:
from sklearn import datasets, ensemble
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [6]:
wordvec_df=gradientmodel()
train_df=wordvec_df
x=train_df
target_df=df1["Indicator"]
y=target_df

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


(50, 100)


In [7]:
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.5)
# print(x_training_data)
print(x_test_data)
# print(y_training_data)
print(y_test_data)

          0         1         2         3         4         5         6   \
16 -0.031750 -0.165393 -0.063892  0.223427 -0.184531  0.039789  0.087326   
40 -0.036628 -0.178954 -0.081839  0.225662 -0.177020  0.057020  0.082684   
25 -0.032898 -0.181955 -0.060466  0.215362 -0.184004  0.040459  0.073999   
42 -0.036485 -0.178939 -0.081701  0.225218 -0.176642  0.056809  0.082460   
46 -0.037283 -0.188793 -0.085202  0.230476 -0.172689  0.058773  0.083873   
4  -0.033975 -0.180698 -0.077421  0.232476 -0.181674  0.046822  0.080951   
44 -0.034225 -0.173666 -0.079974  0.216560 -0.161604  0.055475  0.075592   
3  -0.040758 -0.185844 -0.068520  0.228680 -0.179047  0.035842  0.079590   
19 -0.028582 -0.172913 -0.064459  0.206998 -0.169708  0.045127  0.079046   
20 -0.028582 -0.172913 -0.064459  0.206998 -0.169708  0.045127  0.079046   
22 -0.036078 -0.195108 -0.075694  0.239113 -0.187989  0.040592  0.092160   
49 -0.034344 -0.171718 -0.079291  0.212745 -0.168440  0.051515  0.075622   
17 -0.032149

In [8]:
gbclf = ensemble.GradientBoostingClassifier()
gbclf.fit(x_training_data, y_training_data)
predictions_lg= gbclf.predict(x_test_data)
predictions_lg

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0])

In [9]:
yprd_df=pd.DataFrame(predictions_lg,index=list(x_test_data.index.values))
yprd_df=yprd_df.rename(columns={0:"GB_Predicted_Indicator"})
yprd_df

,GB_Predicted_Indicator
16,0
40,1
25,1
42,1
46,1
4,0
44,1
3,0
19,0
20,0


In [10]:
# df1=df1.join(yprd_df)
# df1.to_csv("gb_prediction.csv",index=False)


final_df=df1.iloc[list(x_test_data.index.values),:]
final_df=pd.concat([final_df,yprd_df],axis=1)
final_df
final_df.to_csv("gb_prediction.csv",index=False)

In [11]:
gbclf.score(x_test_data, y_test_data)

0.36

In [13]:
# y_pred = reg.predict(x_test_data)
# y_pred

In [14]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(x_training_data, y_training_data)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(x_training_data, y_training_data)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(x_test_data, y_test_data)))
#     print("Accuracy score (prediction): {0:.3f}".format(gb_clf.score(y_test_data, y_pred)))

Learning rate:  0.05
Accuracy score (training): 0.760
Accuracy score (validation): 0.360
Learning rate:  0.075
Accuracy score (training): 0.760
Accuracy score (validation): 0.360
Learning rate:  0.1
Accuracy score (training): 0.760
Accuracy score (validation): 0.360
Learning rate:  0.25
Accuracy score (training): 0.760
Accuracy score (validation): 0.360
Learning rate:  0.5
Accuracy score (training): 0.760
Accuracy score (validation): 0.360
Learning rate:  0.75
Accuracy score (training): 0.760
Accuracy score (validation): 0.360
Learning rate:  1
Accuracy score (training): 0.760
Accuracy score (validation): 0.360


In [15]:
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.5, max_features=2, max_depth = 2, random_state = 0)
gb.fit(x_training_data, y_training_data)
predictions = gb.predict(x_test_data)

print("Confusion Matrix:")
print(confusion_matrix(y_test_data, predictions))
print()
print("Classification Report")
print(classification_report(y_test_data, predictions))

Confusion Matrix:
[[ 5  5]
 [11  4]]

Classification Report
              precision    recall  f1-score   support

           0       0.31      0.50      0.38        10
           1       0.44      0.27      0.33        15

    accuracy                           0.36        25
   macro avg       0.38      0.38      0.36        25
weighted avg       0.39      0.36      0.35        25

